# シェル

In [1]:
import "os";
import "os/exec";

## 通常の実行

In [2]:
c:=exec.Command("ls","-a");
c.Start()
c.Wait()

## 実行方法をカスタマイズ
シェルを実行するためのいくつかの関数があり,ここではそれぞれを見ていく  

* `exec.Command(cmd,args...)` → `Cmd*`
	コマンド名 `cmd` と引数 `args` を指定していく

* `Cmd.Env:[]string`  
	環境変数の配列  
    `[]string{"key1=value1","key2=value2"...}` のように指定する

* `Cmd.Dir:string`  
	カレントディレクトリ

* `Cmd.Stdin:io.Reader`
* `Cmd.Stdout:io.Writer`
* `Cmd.Stderr:io.Writer`
	IOの入出力先
	* `nil`,`os.DevNull` : 出力を破棄/空の入力
	* `os.Stdout`,`os.Stderr`,`os.Stdin` : 親プロセスの入出力を継承
	* `io.Reader`,`io.Writer` : 指定のファイルで読み書き/別プロセスへのパイプ
	* 指定しない : `Stdin/out/errPipe` を介して読み書き
    
* `ProcessState:os.ProcessState`
	プロセスの状態
	* `ProcessState.Exited():bool` : 終了したかどうか
	* `ProcessState.Sucess():bool` : 成功したかどうか
	* `ProcessState.ExitCode():int` : 終了コード
	* `ProcessState.Pid():int` : プロセスID

### 実行
* `Cmd.Output()` → `byte[],error` : 実行を開始し,実行が完了するのを待って,stdoutを返す
* `Cmd.CombinedOutput()` → `byte[],error` : 実行を開始し,実行が完了するのを待って,stdout,stderrを返す
* `Cmd.Run()` → `error` : 実行を開始し,実行が完了するのを待つ
* `Cmd.StdinPipe()` → `io.WriteCloser,error` : stdinの書き込みハンドル  
	別プロセスの `.Stdout`,`.Stderr` に渡してパイプしたり,次のようにstdinに書き込むこともできる
	```Go
	stdin,err:=c.StdinPipe()
	os.WriteString(stdin,"stdinの内容")
	os.Close() // .Close() しないと入力が完了しない
	```
* `Cmd.StdoutPipe()`,`Cmd.StderrPipe()` → `io.ReadCloser,error` : stdout,stderrの書き込みハンドル  
	別プロセスの `.Stdin` に渡してパイプしたり,次のようにstdout,stderrから読み込むこともできる
	```Go
	stdout,err:=c.StdoutPipe()
	text,err:=ioutil.ReadAll(stdout)
	text // stdoutの内容
	```
* `Cmd.Start()` → `error` : 実行を開始する  
	完了を待つために `.Wait()` が必要
* `Cmd.Wait()` → `error` : 実行が完了するのを待つ